In [1]:
from pyspark.sql import SparkSession

In [2]:
spark= SparkSession.builder.appName('Mlib').getOrCreate()

22/04/02 22:47:51 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.195 instead (on interface wlp0s20f3)
22/04/02 22:47:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/02 22:47:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/02 22:47:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/02 22:47:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark

In [7]:
df = spark.read.csv('data5.csv',header=True,inferSchema=True)
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [9]:
from pyspark.ml.feature import VectorAssembler
group = VectorAssembler(inputCols=['age','Experience'],outputCol='New')

In [10]:
df1 = group.transform(df)

In [11]:
df1.show()

+---------+---+----------+------+-----------+
|     Name|age|Experience|Salary|        New|
+---------+---+----------+------+-----------+
|    Krish| 31|        10| 30000|[31.0,10.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0]|
|     Paul| 24|         3| 20000| [24.0,3.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0]|
+---------+---+----------+------+-----------+



In [12]:
df2=df1.select('New','Salary')
df2.show()

+-----------+------+
|        New|Salary|
+-----------+------+
|[31.0,10.0]| 30000|
| [30.0,8.0]| 25000|
| [29.0,4.0]| 20000|
| [24.0,3.0]| 20000|
| [21.0,1.0]| 15000|
| [23.0,2.0]| 18000|
+-----------+------+



In [25]:
from pyspark.ml.regression import LinearRegression
train,test = df2.randomSplit([0.9,0.1])
final = LinearRegression(featuresCol='New',labelCol='Salary')
final = final.fit(train)

22/04/02 23:11:03 WARN Instrumentation: [9987053b] regParam is zero, which might cause numerical instability and overfitting.


In [26]:
final.coefficients

DenseVector([-115.8558, 1602.7301])

In [27]:
final.intercept

16904.795239761766

In [21]:
trainCheck = final.evaluate(test)


In [28]:
trainCheck.predictions.show()

+----------+------+------------------+
|       New|Salary|        prediction|
+----------+------+------------------+
|[23.0,2.0]| 18000|17275.132275132273|
|[30.0,8.0]| 25000| 26878.30687830688|
+----------+------+------------------+



/home/narco/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
